In [ ]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.sparse import coo_matrix
from time import time
import matplotlib.pyplot as plt

#-------------------------------------------------------------------------------
# SECTION 1.1 - DENSE NOTATION: Ex 1. Compute the PR vector of $M_{m}$ using 
# the power method (adapted to PR computation). The algorithm reduces to iterate 
# $x_{k+1} = (1 − m)GDx_{k} + ez^{t}x_{k}$ until $||x_{k+1} − x_{k}||_{∞} < tol$. 
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- problem_gen_dense(G, m=0.15)
- PageRank_dense(G, m=0.15, tol=1e-12, output=True)
These have been used to obtain the presented outputs
'''

def problem_gen_dense(G, m=0.15):
    """ problem_gen
    Function to generate the necessary vectors and matrices associated to the link matrix G
    Returns M, mS, e, z, A, D, n
    """
    
    # Retrieve the dimension of the problem ------------------------------------
    n_pages = G.shape[0]
    
    # Compute the vector n and the matrix D ------------------------------------
    n = np.sum(G, axis=0)    # array containing the out_degree for each column j of G
    D = np.zeros([n_pages, n_pages])
    for j in range(n_pages):
        if n[j] !=0:
            D[j,j] = 1/n[j]
    
    # Compute the matrix A -----------------------------------------------------
    A = np.dot(G, D)
    
    # Compute the product mS ---------------------------------------------------
    e = np.ones([n_pages, 1])
    z = 1/n_pages*np.ones([n_pages, 1])
    for j in range(n_pages):
        if A[:,j].any != 0:
            z[j] = m/n_pages
    mS = np.dot(e, z.T)
    
    # Compute the matrix M -----------------------------------------------------
    M = (1-m)*A + mS
    
    return M, mS, e, z, A, D, n

def PageRank_dense(G, m=0.15, tol=1e-12, output=True):
    """ PageRank
    Computes the page rank of page k in the direct graph defined by the link matrix G
    Returns the page rank vector PR
    """
    
    start = time()
    
    # Retrieve the dimension of the problem ------------------------------------
    n_pages = G.shape[0]
    
    # Generate the necessary vectors and matrices associated to the link matrix G 
    M, mS, e, z, A, D, n = problem_gen_dense(G, m)        # This is not optimal but since in Ex.2 we are optimizing the method in order to 
                                                    # handle larger dataset, this method is written this way for clarity 
    
    # Initialize xk and xk_1 ---------------------------------------------------
    xk = np.random.rand(n_pages)
    xk_1 = xk+1     # Initialized in order to enter the while loop, updated as soon as it enters the loop

    # Power method -------------------------------------------------------------
    while (norm(xk_1-xk, ord=np.inf)>tol):
        xk = xk_1          # Update the value of xk
        xk_1 = np.dot(M, xk)
    
    # Visualize the results ----------------------------------------------------
    end = time()
    if output == True:
        print("Computation time:", end-start)
        print("Computed Page Rank vector:", xk_1 / np.sum(xk_1))
    
    return xk_1 / np.sum(xk_1), end-start

#-------------------------------------------------------------------------------
# SECTION 1.2 - SPARSE NOTATION: Ex 1. Compute the PR vector of $M_{m}$ using 
# the power method (adapted to PR computation). The algorithm reduces to iterate 
# $x_{k+1} = (1 − m)GDx_{k} + ez^{t}x_{k}$ until $||x_{k+1} − x_{k}||_{∞} < tol$.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- problem_gen(G, m=0.15, mode=None)
- PageRank(G, m=0.15, tol=1e-12, mode=None, output=True)
- computation_time(G, M, mode, output=False)
These have been used to obtain the presented outputs
'''

def problem_gen(G, m=0.15, mode=None):
    """ problem_gen
    Function to generate the necessary vectors and matrices associated to the link matrix G
    Returns M, mS, e, z, A, D, n
    """
    
    if mode == None:
        print('Specify a mode')
        return
    
    elif mode == 'dense':
        
        # Retrieve the dimension of the problem --------------------------------
        n_pages = G.shape[0]

        # Compute the vector n and the matrix D --------------------------------
        n = np.sum(G, axis=0)    # array containing the out_degree for each column j of G
        D = np.zeros([n_pages, n_pages])
        for j in range(n_pages):
            if n[j] !=0:
                D[j,j] = 1/n[j]

        # Compute the matrix A -------------------------------------------------
        A = np.dot(G, D)

        # Compute the product mS -----------------------------------------------
        e = np.ones([n_pages, 1])
        z = 1/n_pages*np.ones([n_pages, 1])
        for j in range(n_pages):
            if A[:,j].any != 0:
                z[j] = m/n_pages
        mS = np.dot(e, z.T)

        # Compute the matrix M -------------------------------------------------
        M = (1-m)*A + mS        
        
        return M, mS, e, z, A, D, n
        
    elif mode == 'coo_matrix':
        # Retrieve the dimension of the problem --------------------------------
        n_pages = G.shape[0]

        # Compute the vector n and the matrix D --------------------------------
        n = coo_matrix(np.sum(G, axis=0))    # array containing the out_degree for each column j of G
        D = coo_matrix((1/n.data, (n.col, n.col)), shape=(n_pages, n_pages))

        # Compute the matrix A -------------------------------------------------
        A = coo_matrix(G.dot(D))

        # Compute the product mS -----------------------------------------------
        e = np.ones(n_pages)               # The product mS is the product of two dense vectors,
        z = 1/n_pages*np.ones(n_pages)         # in this case the sparse notation is not called for
        for j in A.col:           
            z[j] = m/n_pages         # zj is m/n for the columns j where A contains non-zero values
        # mS = np.dot(e, z.T)        # In this case if we build mS we run into the same problems as storing the matrices as dense

        # Compute the matrix M -------------------------------------------------
        # M = (1-m)*A + mS           # Thus we cannot compute M explicitly, we will compute mS in the PageRank method to avoid computing the matrix
        
        return e, z, A, D, n

def PageRank(G, m=0.15, tol=1e-12, mode=None, output=True):
    """ PageRank
    Computes the page rank of page k in the direct graph defined by the link matrix G
    Returns the page rank vector PR
    """
    
    # Retrieve the dimension of the problem ------------------------------------
    n_pages = G.shape[0]
    
    # Initialize xk and xk_1 ---------------------------------------------------
    xk = np.random.rand(n_pages)
    xk_1 = xk+1     # Initialized in order to enter the while loop, updated as soon as it enters the loop
    
    if mode == None:
        print('Specify a mode')
        return
    
    elif mode == 'dense':
        
        start = time()
        
        # Generate the necessary vectors and matrices associated to the link matrix G 
        M, mS, e, z, A, D, n = problem_gen(G, m, mode)        # This is not optimal but since in Ex.2 we are optimizing the method in order to 
                                                              # handle larger matrices, this method is written this way for clarity 
        # Power method ---------------------------------------------------------
        while (norm(xk_1-xk, ord=np.inf)>tol):
            xk = xk_1          # Update the value of xk
            xk_1 = np.dot(M, xk)
            
        # Visualize the results ------------------------------------------------
        end = time()
        if output == True:
            print("Computation time:", end-start)
            print("Computed Page Rank vector:", xk_1 / np.sum(xk_1))
        
        return xk_1 / np.sum(xk_1), end-start
    
    elif mode == 'coo_matrix':
        
        start = time()
        
        # Generate the necessary vectors and matrices associated to the link matrix G 
        e, z, A, D, n = problem_gen(G, m, mode)        # This is not optimal but since in Ex.2 we are optimizing the method in order to 
                                                              # handle larger matrices, this method is written this way for clarity 
        # Power method ---------------------------------------------------------
        while (norm(xk_1-xk, ord=np.inf)>tol):
            xk = xk_1          # Update the value of xk
            xk_1 = (1-m)*A.dot(xk) + e*(np.dot(z,xk))       # Here the product M*xk is computed explicitly, otherwise the contribute mS would give a dense matrix
            
        # Visualize the results ------------------------------------------------
        end = time()
        if output == True:
            print("Computation time:", end-start)
            print("Computed Page Rank vector:", xk_1 / np.sum(xk_1))
            
        return xk_1 / np.sum(xk_1), end-start

def computation_time(G, M, mode, output=False):
    
    comp_time = np.zeros(len(M))
    
    for i in range(len(M)):
        PR, comp_time[i] = PageRank(G, m=M[i], mode=mode, output=output)
        
    plt.figure()
    plt.title('Computation time over values of m, mode=' + str(mode))
    plt.plot(M, comp_time)
    
    return

#-------------------------------------------------------------------------------
# SECTION 2.1 - IMPLEMENTATION: Compute the PR vector of $M_{m}$ using the power
# method without storing matrices.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- PageRank(G, m=0.15, tol=1e-12, mode=None, output=True)
These have been used to obtain the presented outputs
'''

def PageRank(G, m=0.15, tol=1e-12, mode=None, output=True):
    """ PageRank
    Computes the page rank of page k in the direct graph defined by the link matrix G
    Returns the page rank vector PR
    """
    
    # Retrieve the dimension of the problem ------------------------------------
    n_pages = G.shape[0]
    
    # Initialize xk and xk_1 ---------------------------------------------------
    xk = np.random.rand(n_pages)
    xk_1 = xk+1     # Initialized in order to enter the while loop, updated as soon as it enters the loop
    
    if mode == None:
        print('Specify a mode')
        return
    
    elif mode == 'dense':
        
        start = time()
        
        # Generate the necessary vectors and matrices associated to the link matrix G 
        M, mS, e, z, A, D, n = problem_gen(G, m, mode)        # This is not optimal but since in Ex.2 we are optimizing the method in order to 
                                                              # handle larger matrices, this method is written this way for clarity 
        # Power method ---------------------------------------------------------
        while (norm(xk_1-xk, ord=np.inf)>tol):
            xk = xk_1          # Update the value of xk
            xk_1 = np.dot(M, xk)
        
        # Visualize the results ------------------------------------------------
        end = time()
        if output == True:
            print("Computation time:", end-start)
            print("Computed Page Rank vector:", xk_1 / np.sum(xk_1))
        
        return xk_1 / np.sum(xk_1), end-start
        
    elif mode == 'coo_matrix':
        
        start = time()
        
        # Generate the necessary vectors and matrices associated to the link matrix G 
        e, z, A, D, n = problem_gen(G, m, mode)        # This is not optimal but since in Ex.2 we are optimizing the method in order to 
                                                              # handle larger matrices, this method is written this way for clarity 
        # Power method ---------------------------------------------------------
        while (norm(xk_1-xk, ord=np.inf)>tol):
            xk = xk_1          # Update the value of xk
            xk_1 = (1-m)*A.dot(xk) + e*(np.dot(z,xk))
        
        # Visualize the results ------------------------------------------------
        end = time()
        if output == True:
            print("Computation time:", end-start)
            print("Computed Page Rank vector:", xk_1 / np.sum(xk_1))
        
        return xk_1 / np.sum(xk_1), end-start
            
    elif mode == 'no_store':
        
        start = time()
        
        # Initialize xk and xk_1 -----------------------------------------------
        x = np.zeros(n_pages)               # Here there is a change of notation made in order to  
        xc = np.ones(n_pages) / n_pages     # be consistent with what is given project statement
        
        # Compute Lj and nj ----------------------------------------------------
        L = []
        n = []
        for j in range(n_pages):     # Go thorugh the column index j
            L.append([])          # Add an empty list for Lj
            L[j] = G.row[np.asarray(G.col==j).nonzero()]       # Lj is the list of row coordinates at indices where the columns coordinates are equal to j
            n.append(len(L[j]))       # Compute nj as the length of Lj 
        
        # Power method ---------------------------------------------------------
        while (norm(x-xc, ord=np.inf)>tol):
            xc=x
            x=np.zeros(n_pages)               # Here there is the change of notation from xk to x and xc,
            for j in range (0,n_pages):       #  only difference from the given code is n_pages (instead of n)
                if(n[j]==0):
                    x=x+xc[j]/n_pages
                else:
                    for i in L[j]:
                        x[i]=x[i]+xc[j]/n[j]
            x=(1-m)*x+m/n_pages
        
        # Visualize the results ------------------------------------------------
        end = time()
        if output == True:
            print("Computation time:", end-start)
            print("Computed Page Rank vector:", x / np.sum(x))
        
        return x / np.sum(x), end-start

#-------------------------------------------------------------------------------
# SECTION 2.2 - COMPARISON: Compute the PR vector of $M_{m}$ using the power
# method without storing matrices.
#-------------------------------------------------------------------------------

'''
In this section the following functions are defined:
- compare(G, M, mode=None, output=False)
These have been used to obtain the presented outputs
'''

def compare(G, M, mode=None, output=False):
    
    comp_time_store = np.zeros(len(M))
    comp_time_no_store = np.zeros(len(M))
    PR_difference = np.zeros(len(M))
    
    for i in range(len(M)):
        PR_store_i, comp_time_store[i] = PageRank(G, m=M[i], mode='coo_matrix', output=output)
        PR_no_store_i, comp_time_no_store[i] = PageRank(G, m=M[i], mode='no_store', output=output)
        PR_difference[i] = norm(PR_store_i-PR_no_store_i, ord=np.inf)
        
    plt.figure()
    plt.title('Computation time over values of m, mode=coo_matrix')
    plt.plot(M, comp_time_store)
    
    plt.figure()
    plt.title('Computation time over values of m, mode=no_store')
    plt.plot(M, comp_time_no_store)
    
    plt.figure()
    plt.title('Norm of difference between the two methods over values of m')
    plt.plot(M, PR_difference)
    
    return